In [1]:

import torch

import bittensor as bt
import pandas as pd
       
from new_agent import HumanAgent
from prompting.tasks import DebuggingTask, QuestionAnsweringTask, SummarizationTask
from tasks.dataset import WikiDataset, StackOverflowDataset, CodingDataset
from transformers import pipeline

def make_agent(task, llm_pipeline):
    bt.logging.info("Creating agent...")
    agent = HumanAgent(
        task=task,
        llm=llm_pipeline,
        system_template=None,
        begin_conversation=True
    )
    bt.logging.info(f'Agent created with persona: {agent.persona}')
    bt.logging.info(f'Agent challenge: {agent.challenge}')
    return agent


/home/ubuntu/steffen/text-prompting-private/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

model_id = "HuggingFaceH4/zephyr-7b-beta"
bt.logging.info("🤖 Loading LLM model...")
llm_pipeline = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",
)

2024-01-08 04:43:42.321 |       INFO       | 🤖 Loading LLM model...        


Loading checkpoint shards: 100%|██████████| 8/8 [00:03<00:00,  2.67it/s]


In [3]:
dataset = WikiDataset()

In [4]:
dataset.next(subset=True, chunk_sep='. ', n_chunks=2)

2024-01-08 04:43:49.157 |       INFO       | Choosing 2 chunks starting at index 1.


{'title': 'Availability-based tariff',
 'url': 'https://en.wikipedia.org/wiki/Availability-based_tariff',
 'length': 35495,
 'extract': '<p class="mw-empty-elt">\n\n</p>\n<p><b>Availability Based Tariff</b> (ABT) is a frequency based pricing mechanism applicable in India for unscheduled electric power transactions. The ABT falls under electricity market mechanisms to charge and regulate power to achieve short term and long term network stability as well as incentives and dis-incentives to grid participants against deviations in committed supplies as the case may be.</p>\n\n\n<h2><span id="Introduction">Introduction</span></h2>\n<p>ABT Mechanism in Electricity sector in India is adopted since the year 2000 and in a few other countries for pricing bulk power across various stakeholders. ABT concerns itself with the tariff structure for bulk power and is aimed at bringing about more responsibility and accountability in power generation and consumption through a scheme of incentives and di

In [5]:
context = dataset.next()
task = SummarizationTask(llm_pipeline=llm_pipeline, context=context)
bt.logging.info(f'Task query: {task.query}')
bt.logging.info(f'Task reference: {task.reference}')

task

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 context = dataset.next()                                                                     │
│   2 task = SummarizationTask(llm_pipeline=llm_pipeline, context=context)                         │
│   3 bt.logging.info(f'Task query: {task.query}')                                                 │
│   4 bt.logging.info(f'Task reference: {task.reference}')                                         │
│                                                                                                  │
│ /home/ubuntu/steffen/text-prompting-private/new_prototype/tasks/dataset.py:243 in next           │
│                                                                                                  │
│   240 │   │   elif subset:                                                                       │
│   241 │   │   │   text = chunk(info['text'], sep=chunk_sep, n_chunks=n_chunks)                   │
│   242 │   │                                                                                      │
│ ❱ 243 │   │   info['text'] = text                                                                │
│   244 │   │   return info                                                                        │
│   245                                                                                            │
│   246                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
UnboundLocalError: local variable 'text' referenced before assignment

In [6]:
make_agent(task, llm_pipeline)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 make_agent(task, llm_pipeline)                                                               │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'task' is not defined

In [7]:
context = dataset.next(subset=True, chunk_sep='\n', n_chunks=5)
task = QuestionAnsweringTask(llm_pipeline=llm_pipeline, context=context)
bt.logging.info(f'Task query: {task.query}')
bt.logging.info(f'Task reference: {task.reference}')

task

2024-01-08 04:43:52.787 |       INFO       | Choosing 5 chunks starting at index 1.
2024-01-08 04:43:52.787 |       INFO       | 🤖 Generating query...         


/home/ubuntu/steffen/text-prompting-private/env/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


2024-01-08 04:43:54.546 |       INFO       | 🤖 Generating reference...     
2024-01-08 04:44:02.100 |       INFO       | Task query: #Question:
In Inazuma Eleven GO 2: Chrono Stone, how does the concept of time travel play a role in the story, and how does it impact the gameplay?


QuestionAnsweringTask(desc='get help on answering a question', goal='to get the answer to the following question', query='#Question:\nIn Inazuma Eleven GO 2: Chrono Stone, how does the concept of time travel play a role in the story, and how does it impact the gameplay?', topic='Inazuma Eleven GO (video game)', subtopic='2011 video games', tags=['2011 video games', 'Association football video games', 'S1 Japanese-language sources (ja)', 'mmunism in fiction', 'Inazuma Eleven video games', 'Level-5 (company) games', 'Nintendo 3DS-only games', 'Nintendo 3DS eShop games', 'Nintendo 3DS games', 'Nintendo games', 'Role-playing video games', 'Short description is different from Wikid', 'Short description matches Wikid', 'Video games developed in Japan', 'Video games with alternative versions'], reward_types=['SIMILARITY_STACK'], reward_threshold=0.5, reference="In Inazuma Eleven GO 2: Chrono Stone, the concept of time travel is introduced as a major plot element. The story follows the protago

2024-01-08 04:44:02.101 |       INFO       | Task reference: In Inazuma Eleven GO 2: Chrono Stone, the concept of time travel is introduced as a major plot element. The story follows the protagonist, Arion Sherwind, as he travels back in time to prevent a catastrophic event that threatens the future of soccer. Arion is aided by his childhood friend, Tenma Matsukaze, who also happens to be the main character from the previous game, Inazuma Eleven GO.

As Arion and Tenma travel through time, they encounter various versions of their friends from Raimon High School and the Inazuma Japan National Team. Some of these versions have been altered by the events of the future, causing them to have different personalities and abilities. Arion and Tenma must work together to set things right and ensure that history unfolds as it should.

The concept of time travel has a significant impact on the gameplay of Inazuma Eleven GO 2: Chrono Stone. Players can travel back in time to replay certain matches

In [8]:
make_agent(task, llm_pipeline)

2024-01-08 04:44:02.112 |       INFO       | Creating agent...             
2024-01-08 04:44:02.113 |       INFO       | 🤖 Generating challenge query...
Cleaning up response: Assistant: "In Inazuma Eleven GO 2: Chrono Stone, can you explain how the time travel mechanism works and how it allows players to manipulate the game's narrative and gameplay elements?"
2024-01-08 04:44:03.481 |       INFO       | Agent created with persona: Persona(profile='scientist', mood='an impatient', tone='indifferent')


This is a roleplaying game where you are impersonating an impatient human user. As a human, you are using AI assistant to get help on answering a question related to Inazuma Eleven GO (video game) (2011 video games) in a indifferent tone.

        Your singular focus is to use the assistant to to get the answer to the following question: #Question:
In Inazuma Eleven GO 2: Chrono Stone, how does the concept of time travel play a role in the story, and how does it impact the gameplay?

2024-01-08 04:44:03.482 |       INFO       | Agent challenge:  "In Inazuma Eleven GO 2: Chrono Stone, can you explain how the time travel mechanism works and how it allows players to manipulate the game's narrative and gameplay elements?"


In [9]:
coding_dataset = CodingDataset(languages=['Python'])


In [10]:

context = coding_dataset.next()
context


{'code': "from django.apps import AppConfig\n\n\nclass UsermessagesConfig(AppConfig):\n    name = 'usermessages'\n",
 'repo_name': 'mikaicebe/xxwctreehole',
 'path': 'nw/wctreehole/usermessages/apps.py',
 'language': 'Python',
 'license': 'mit',
 'size': 99}

In [11]:

db_task = DebuggingTask(llm_pipeline=llm_pipeline, context=context)
db_task

2024-01-08 04:44:59.416 |       INFO       | Removing the following 1 chunks: ['s'] at indices [49]
2024-01-08 04:44:59.416 |       INFO       | Swapping chunk '\n' at index 97 with chunk 'C' at index 53


DebuggingTask(desc='get help with debugging', goal='ask for help fixing the broken piece of code', query="from django.apps import AppConfig\n\n\nclass Usermesages\nonfig(AppConfig):\n    name = 'usermessages'C", topic='mikaicebe/xxwctreehole', subtopic='nw/wctreehole/usermessages/apps.py', tags='Python', reward_types=['SIMILARITY_STACK'], reward_threshold=0.5, reference="from django.apps import AppConfig\n\n\nclass UsermessagesConfig(AppConfig):\n    name = 'usermessages'\n", criteria=('',), delimiter='', complete=False)

In [12]:
print(db_task.reference)

from django.apps import AppConfig


class UsermessagesConfig(AppConfig):
    name = 'usermessages'



In [13]:
print(db_task.query)

from django.apps import AppConfig


class Usermesages
onfig(AppConfig):
    name = 'usermessages'C


In [14]:
make_agent(db_task, llm_pipeline)

2024-01-08 04:44:59.435 |       INFO       | Creating agent...             
2024-01-08 04:44:59.435 |       INFO       | 🤖 Generating challenge query...
2024-01-08 04:45:02.033 |       INFO       | Agent created with persona: Persona(profile='student', mood='a confused', tone='unfriendly')
2024-01-08 04:45:02.033 |       INFO       | Agent challenge: How can I fix the error in the Usermessages AppConfig class in the mikaicebe/xxwctreehole repository? It seems to be causing a problem with the django.apps import AppConfig line. Please provide detailed instructions on how to resolve this issue. Your response should be in a clear and concise manner, with step-by-step guidance that a novice programmer can follow.


This is a roleplaying game where you are impersonating a confused human user. As a human, you are using AI assistant to get help with debugging related to mikaicebe/xxwctreehole (nw/wctreehole/usermessages/apps.py) in a unfriendly tone.

        Your singular focus is to use the assistant to ask for help fixing the broken piece of code: from django.apps import AppConfig


class Usermesages
onfig(AppConfig):
    name = 'usermessages'C